In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch

from train_configurations import (regnet_y_400mf, tracknet_v2, tracknet_v2_2f,
                                  tracknet_v2_4f, tracknet_v2_6f,
                                  tracknet_v2_rnn, tracknet_v2_rnn_scheduler)

from train_configurations.utils import get_standard_test_dataset
from detection.testing import _get_checkpoint_filename, _get_results_folder

# figures_folder = 'figures'
figures_folder = '../TFM/pictures'

if not os.path.exists(figures_folder):
    os.makedirs(figures_folder)

plt.style.use('default')

%load_ext autoreload
%autoreload 2

# Examples for training data

**Visualization of input frames and heatmap**

In [ ]:
# load model from the desired train configuration
dataset = get_standard_test_dataset(tracknet_v2, 'prova')

Compute heatmap for an example frame

In [ ]:
dataset_element = 15

with torch.no_grad():
    frames, heatmap = dataset[dataset_element]
    frames = frames.to(torch.float32)

    heatmap_pred = model(frames.unsqueeze(0)).squeeze().numpy()

    heatmap = heatmap.to(torch.float32).squeeze().numpy()
    frames = frames.numpy().transpose(1, 2, 0)

frames = [frames[:,:,3*i:3*(i+1)] for i in range(3)]

In [ ]:
w, h, dpi = 2*640, 2*360, 100
fig, axs = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi, nrows=2, ncols=2)
axs = axs.ravel()

for i, (frame, ax) in enumerate(zip(frames, axs)):
    ax.set_title(f'Frame {i+1}')
    ax.imshow(frame)

    x = dataset._label_df['x'][i+dataset_element]
    y = dataset._label_df['y'][i+dataset_element]
    ax.scatter(x*frame.shape[1], y*frame.shape[0], zorder=100, facecolors='none', edgecolors='y', linewidths=3, s=150)

axs[3].imshow(heatmap, cmap='gray')
axs[3].set_title('Target heatmap')

fig.tight_layout()

fig.savefig(os.path.join(figures_folder, 'sample_input_a.png'))

plt.show()

In [ ]:
w, h, dpi = 640, 360, 100
fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)

ax.imshow(frames[-1], cmap='gray')
ax.imshow(heatmap, cmap='gray', alpha=0.5)

ax.set_axis_off()
fig.tight_layout(pad=0)

fig.savefig(os.path.join(figures_folder, 'sample_input_b.png'))

plt.show()

# Loss histories

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from detection.visualization import show_loss_history, get_loss_history

In [ ]:
from train_configurations import (regnet_y_400mf, tracknet_v2, tracknet_v2_2f,
                                  tracknet_v2_4f, tracknet_v2_6f,
                                  tracknet_v2_rnn, tracknet_v2_rnn_scheduler)

## RegNet and Standard architectures

In [ ]:
training_configurations = [regnet_y_400mf, tracknet_v2, tracknet_v2_2f, tracknet_v2_6f] #, tracknet_v2_rnn_scheduler]
names = ['regnet_y_400mf', 'tracknet_3f', 'tracknet_2f', 'tracknet_6f'] #, 'tracknet_rnn']
titles = ['RegNet, $n=3$', 'Standard, $n=3$', 'Standard, $n=2$', 'Standard, $n=6$'] #, 'Recurrent, $n=1$, $h=3$']

for training_configuration, name, title in zip(training_configurations, names, titles):
    w, h, dpi = 640, 480, 120
    fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)

    ax.set_yscale('log')
    ax = show_loss_history(training_configuration, ax=ax)

    fig.suptitle(title)
    fig.tight_layout

    fig.savefig(os.path.join(figures_folder, f'{name}.pdf'))
    plt.show()

In [ ]:
import pandas as pd

for training_configuration, name in zip(training_configurations, names):
    print(name)
    print()

    train_loss, val_loss = get_loss_history(training_configuration)
    df = pd.DataFrame({'epoch': np.arange(len(train_loss))+1, 'train loss': train_loss, 'validation loss': val_loss})
    print(df.to_latex(index=False))

## Recurrent architecture

In [ ]:
def plot_rnn_history(training_configuration):
    phases = np.array([0, 10, 15, 20, 25, 30, 35]) + 0.5

    w, h, dpi = 640, 480, 120
    fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)

    ax = show_loss_history(training_configuration, ax=ax, epoch_range=(1, 35))

    xlim = ax.get_xbound()
    ylim = ax.get_ybound()

    for i, p in enumerate(phases[1:-1]):
        ax.plot([p,p], [ylim[0], ylim[1]], 'k--', linewidth=0.5 if i%2==0 else 0.7)

    ax.set_ylim(ylim)

    bbox = {'boxstyle': 'round',
            'facecolor': 'w',
            'edgecolor': 'None',
            'alpha': 0.7}

    for i, c in enumerate(['r', 'g', 'b']):
        t = 0.1
        xpos = 10*i+6 if i==0 else 10*i+8
        ax.annotate(f'Phase {i+1}', [xpos, t*ylim[1]+(1-t)*ylim[0]], bbox=bbox)
        ax.fill_between([phases[2*i], phases[2*(i+1)]], ylim[0], ylim[1], color=c, alpha=0.15)

    ax.legend(loc='upper right')

    return fig, ax

fig, ax = plot_rnn_history(tracknet_v2_rnn)
fig.suptitle('Recurrent, $n=1$, $h=3$, constant lr')
fig.savefig(os.path.join(figures_folder, f'tracknet_rnn.pdf'))
fig.tight_layout()
plt.show()

fig, ax = plot_rnn_history(tracknet_v2_rnn_scheduler)
fig.suptitle('Recurrent, $n=1$, $h=3$, step lr')
fig.savefig(os.path.join(figures_folder, f'tracknet_rnn_scheduler.pdf'))
fig.tight_layout()
plt.show()

# Sample outputs

In [ ]:
def get_trained_model(training_configuration, training_phase=None):
    config = training_configuration.Config()

    checkpoint_folder = config._checkpoint_folder
    if training_phase is not None:
        checkpoint_folder = os.path.join(checkpoint_folder, training_phase)

    results_folder = _get_results_folder(checkpoint_folder, None)
    checkpoint_path = os.path.join(results_folder, _get_checkpoint_filename(checkpoint_folder))

    model = config.get_model()
    model.eval()
    model.load(checkpoint_path, device='cpu')

    return get_standard_test_dataset(training_configuration, 'prova'), model


def get_output(model, dataset, i):
    frame, _ = dataset[i]

    with torch.no_grad():
        output = model(frame.to(torch.float32).unsqueeze(0)).numpy().squeeze()
    return output


def get_frame(dataset, i):
    frame, _ = dataset[i]

    return frame[-3:].to(torch.float32).numpy().transpose(1, 2, 0)

In [ ]:
dataset, model = get_trained_model(tracknet_v2_rnn_scheduler, 'phase_3_1')

In [ ]:
def show_sample_output(training_configuration, frame_offset):
    training_phase = 'phase_3_1' if frame_offset==1 else None
    sample = 19 - frame_offset
    dataset, model = get_trained_model(training_configuration, training_phase)
    frame = get_frame(dataset, sample)
    output =  get_output(model, dataset, sample)

    w, h, dpi = 640, 480, 120
    fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)

    ax.imshow(frame)
    ax.imshow(output, alpha=0.7, cmap='gray')

    ax.set_axis_off()
    fig.tight_layout(pad=0)

    fig.savefig(os.path.join(figures_folder, f"tracknet_{frame_offset}f_example.png"))

    plt.show()

training_configurations = [tracknet_v2_2f, tracknet_v2, tracknet_v2_4f, tracknet_v2_6f, tracknet_v2_rnn, tracknet_v2_rnn_scheduler]

n_frames = [2, 3, 4, 6, 1, 1]

for n, training_configuration in zip(n_frames, training_configurations):
    show_sample_output(training_configuration, n)
